In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras.models as models
import keras.layers as layers
import keras.optimizers as optimizers
from keras.preprocessing.image import ImageDataGenerator

In [5]:
# Discriminator

# The input to the discriminator is a 28x28x1 image.
input = layers.Input(shape=(28, 28, 1))


def discriminator_block(x, filters, kernel_size, strides, dropout=False,  batch_norm=True):
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.3)(x)

    # Add a dropout layer if specified.
    # This is used to prevent overfitting.
    # combining dropout with batch normalization is not recommended.
    if dropout and batch_norm:
        print('WARNING: Batch normalization and dropout are both enabled.')
    if batch_norm:
        x = layers.BatchNormalization(momentum=0.8)(x)
    if dropout:
        x = layers.Dropout(0.25)(x)
    return x


# Add the discriminator blocks.
x = discriminator_block(input, 16, 3, 1, batch_norm=False)
x = discriminator_block(x, 32, 3, 2, dropout=True, batch_norm=False)
x = discriminator_block(x, 64, 3, 2, dropout=True, batch_norm=False)
x = discriminator_block(x, 128, 3, 1)

# The output layer.
x = layers.Flatten()(x)
output = layers.Dense(1, activation='sigmoid')(x)

# Create the model.
discriminator = models.Model(input, output, name='discriminator')

In [9]:
# Generator

# The input to the generator is a 100-dimensional vector.
input = layers.Input(shape=(100,))

def generator_block(x, filters, kernel_size, strides, batch_norm=True):
    x = layers.Dense(filters)(x)
    x = layers.Conv2DTranspose(filters, kernel_size, strides=strides, padding='same')(x)
    if batch_norm:
        x = layers.BatchNormalization(momentum=0.8)(x)
    x = layers.LeakyReLU(alpha=0.3)(x)
    return x

x = layers.Dense(128 * 7 * 7)(input)
x = layers.BatchNormalization(momentum=0.8)(x)
x = layers.LeakyReLU(alpha=0.3)(x)

x = layers.Reshape((7, 7, 128))(x)

# Add the generator blocks.
x = generator_block(x, 128, 3, 1)
x = generator_block(x, 64, 3, 2)
x = generator_block(x, 32, 3, 2)
x = generator_block(x, 16, 3, 1, batch_norm=False)

# The output layer.
output = layers.Conv2DTranspose(1, (5, 5), strides=1, padding='same', activation='tanh')(x)

# Create the model.
generator = models.Model(input, output, name='generator')


In [10]:
# GAN

# The input to the GAN is a 100-dimensional vector.
input = layers.Input(shape=(100,))
# The output of the GAN is the output of the discriminator.
output = discriminator(generator(input))
# Create the model.
gan = models.Model(input, output, name='gan')

# Compile the model.
gan.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(1e-4))

In [20]:
import keras.backend as K
import numpy.random as npr

ModuleNotFoundError: No module named 'tf'

In [17]:
# Load the MNIST dataset.
(x_train, _),  (x_test, _)= tf.keras.datasets.mnist.load_data()

train = np.concatenate((x_train, x_test), axis=0)

# Normalize the data to the range [-1, 1].
train = (train.astype(np.float32) - 127.5) / 127.5

print(train.shape)

(70000, 28, 28)


In [ ]:
# training

batch = 32
epochs = 100


for epoch in range(epochs):